## Running the Supervised Optimizer

From start to finish, on pretrained weights


In [1]:
import torch
import numpy as np
import os
from Hamiltonian import Ising
from model import TransformerModel
from optimizer_supervised import Optimizer

In [2]:
system_sizes = np.arange(4, 10, 2).reshape(-1, 1)
Hamiltonians = [Ising(size, periodic=True) for size in system_sizes]
param_dim = Hamiltonians[0].param_dim
embedding_size = 32
n_head = 8
n_hid = embedding_size
n_layers = 8
dropout = 0
minibatch = 10000
param_range = None
point_of_interest = None
use_SR = False

print("Sizes:\n", system_sizes)
print("Hamiltonians:", Hamiltonians)
print("Dimensions of parameter space:", param_dim)
print("Number of units in a feedforward layer:", n_hid)

Sizes:
 [[4]
 [6]
 [8]]
Hamiltonians: [<Hamiltonian.Ising object at 0x00000167EF1BE2B0>, <Hamiltonian.Ising object at 0x00000167E2CB6220>, <Hamiltonian.Ising object at 0x00000167EF1BE3D0>]
Dimensions of parameter space: 1
Number of units in a feedforward layer: 32


c:\Users\csmuser\Desktop\Spandan_Suthar_Research\tqs\Hamiltonian_utils.py:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  system_size = torch.tensor(system_size, dtype=torch.int64).reshape(-1)


In [3]:
testmodel = TransformerModel(
    system_sizes,
    param_dim,
    embedding_size,
    n_head,
    n_hid,
    n_layers,
    dropout=dropout,
    minibatch=minibatch,
)

results_dir = "results"
paper_checkpoint_name = "ckpt_100000_Ising_32_8_8_0.ckpt"
paper_checkpoint_path = os.path.join(results_dir, paper_checkpoint_name)
checkpoint = torch.load(paper_checkpoint_path)
testmodel.load_state_dict(checkpoint)

c:\Users\csmuser\anaconda3\envs\ian_p_env\lib\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer was not TransformerEncoderLayer
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


<All keys matched successfully>

In [4]:
opt = Optimizer(testmodel, Hamiltonians, point_of_interest=point_of_interest)

In [5]:
epochs = 1000
param_range = torch.tensor([[0.5, 1.5]])
param_step = torch.tensor([0.05])

opt.train(epochs=epochs, param_range=param_range, param_step=param_step, start_iter=0)

System size: tensor([4], dtype=torch.int32), Hamiltonian time: 0.5738997459411621, Last loss: 0.03893289342522621
System size: tensor([6], dtype=torch.int32), Hamiltonian time: 1.2855117321014404, Last loss: 0.008415862917900085
System size: tensor([8], dtype=torch.int32), Hamiltonian time: 2.1124184131622314, Last loss: 0.0019499878399074078
Epoch time: 3.971829891204834
System size: tensor([4], dtype=torch.int32), Hamiltonian time: 0.4702792167663574, Last loss: 0.030648335814476013
System size: tensor([6], dtype=torch.int32), Hamiltonian time: 1.2737936973571777, Last loss: 0.007754218298941851
System size: tensor([8], dtype=torch.int32), Hamiltonian time: 2.116270065307617, Last loss: 0.0019481540657579899
Epoch time: 3.8603429794311523


KeyboardInterrupt: 

In [ ]:
H = Hamiltonians[2]
param = torch.tensor([0.8000])
ground_energy, psi_true = H.calc_ground(param)
psi_true

tensor([-0.4928,  0.1036,  0.1036,  ...,  0.1036,  0.1036, -0.4928],
       dtype=torch.float64)

In [ ]:
basis = H.generate_basis()
basis.shape

torch.Size([14, 16384])

In [ ]:
from model_utils import compute_psi

testmodel.set_param(system_size=H.system_size, param=param)
log_amp, log_phase = compute_psi(testmodel, basis, H.symmetry, check_duplicate=True)
print(log_amp)
print(log_phase)

tensor([-2.0506, -3.7310, -4.7868,  ..., -4.7868, -3.7310, -2.0506],
       grad_fn=<LogBackward0>)
tensor([-6.0865,  0.1709,  0.1618,  ...,  0.1618,  0.1709, -6.0865],
       grad_fn=<MulBackward0>)
